# Project 1: "Projectile motion of a baseball under air resistance"
In this project, we study the motion of a baseball under the influence of gravity and air resistance. We shall investigate how the range of a fast-moving baseball is affected by air resistance.

## Section 0. Environment Setup
Here, we install all necessary packages for experiment.

**Package Installation**

In [ ]:
import numpy as np

## Section 1. Problem Investigation
In this section, we will invesigate the motion of the projectile motion step by step.

**1. The equation of motion and the variable**

We declare necessary function that calculates the motion, and relevants variables.

In [ ]:
def ProjectileMotile()